In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['Data_Dictionary.xlsx', 'historical_transactions.csv', 'merchants.csv', 'new_merchant_transactions.csv', 'sample_submission.csv', 'test.csv', 'train.csv']


In [2]:
#an attempt to use embedding for tabular data
# thanks to https://www.kaggle.com/artgor/nn-baseline, https://www.kaggle.com/hrmello/starter-neural-network-3-939-lb
# and kaggle kernel on feature engineering like https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-elo and others

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
#from keras_tqdm import TQDMNotebookCallback

import os

import tensorflow as tf
import keras as K
print(tf.__version__)
print(K.__version__)
print(tf.keras.__version__)

SEED = 2018


1.9.0
2.2.0
2.1.6-tf


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.linear_model import Ridge

from sklearn import preprocessing
import warnings
import datetime
warnings.filterwarnings("ignore")
import gc

from scipy.stats import describe
%matplotlib inline

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error


In [6]:
#Loading Train and Test Data
train = pd.read_csv("../input/train.csv", parse_dates=["first_active_month"])
test = pd.read_csv("../input/test.csv", parse_dates=["first_active_month"])
print("{} observations and {} features in train set.".format(train.shape[0],train.shape[1]))
print("{} observations and {} features in test set.".format(test.shape[0],test.shape[1]))

201917 observations and 6 features in train set.
123623 observations and 5 features in test set.


In [7]:
train["month"] = train["first_active_month"].dt.month
test["month"] = test["first_active_month"].dt.month
train["year"] = train["first_active_month"].dt.year
test["year"] = test["first_active_month"].dt.year
train['elapsed_time'] = (datetime.date(2018, 2, 1) - train['first_active_month'].dt.date).dt.days
test['elapsed_time'] = (datetime.date(2018, 2, 1) - test['first_active_month'].dt.date).dt.days
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,month,year,elapsed_time
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,6,2017,245
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,1,2017,396
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,8,2016,549
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,9,2017,153
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,11,2017,92


In [8]:
hist_trans = pd.read_csv("../input/historical_transactions.csv")
hist_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [9]:
hist_trans = pd.get_dummies(hist_trans, columns=['category_2', 'category_3'])
hist_trans['authorized_flag'] = hist_trans['authorized_flag'].map({'Y': 1, 'N': 0})
hist_trans['category_1'] = hist_trans['category_1'].map({'Y': 1, 'N': 0})
hist_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,16,37,1,0,0,0,0,1,0,0
1,1,C_ID_4e6213e9bc,88,0,0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,16,16,1,0,0,0,0,1,0,0
2,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,16,37,1,0,0,0,0,1,0,0
3,1,C_ID_4e6213e9bc,88,0,0,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,16,34,1,0,0,0,0,1,0,0
4,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,16,37,1,0,0,0,0,1,0,0


In [10]:
def aggregate_transactions(trans, prefix):  
    trans.loc[:, 'purchase_date'] = pd.DatetimeIndex(trans['purchase_date']).\
                                      astype(np.int64) * 1e-9
    
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'category_1': ['mean'],
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        'merchant_id': ['nunique'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_date': [np.ptp],
        'month_lag': ['min', 'max']
    }
    agg_trans = trans.groupby(['card_id']).agg(agg_func)
    agg_trans.columns = [prefix + '_'.join(col).strip() 
                           for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)
    
    df = (trans.groupby('card_id')
          .size()
          .reset_index(name='{}transactions_count'.format(prefix)))
    
    agg_trans = pd.merge(df, agg_trans, on='card_id', how='left')
    
    return agg_trans

In [11]:
import gc
merch_hist = aggregate_transactions(hist_trans, prefix='hist_')
del hist_trans
gc.collect()
train = pd.merge(train, merch_hist, on='card_id',how='left')
test = pd.merge(test, merch_hist, on='card_id',how='left')
del merch_hist
gc.collect()
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,month,year,elapsed_time,hist_transactions_count,...,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,hist_installments_mean,hist_installments_max,hist_installments_min,hist_installments_std,hist_purchase_date_ptp,hist_month_lag_min,hist_month_lag_max
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,6,2017,245,260,...,-0.739395,0.212139,4,0.015385,1,0,0.123314,20977987.0,-8,0
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,1,2017,396,350,...,-0.742400,0.384967,543,1.551429,10,-1,1.510777,33717687.0,-12,0
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,8,2016,549,43,...,-0.730138,0.087380,0,0.000000,0,0,0.000000,35635623.0,-13,0
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,9,2017,153,77,...,-0.740897,0.261624,84,1.090909,3,-1,0.588974,13375339.0,-5,0
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,11,2017,92,133,...,-0.746156,1.352094,182,1.368421,12,1,1.896862,9405641.0,-3,0


In [12]:
new_trans = pd.read_csv("../input/new_merchant_transactions.csv")
new_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [13]:
new_trans = pd.get_dummies(new_trans, columns=['category_2', 'category_3'])
new_trans['authorized_flag'] = new_trans['authorized_flag'].map({'Y': 1, 'N': 0})
new_trans['category_1'] = new_trans['category_1'].map({'Y': 1, 'N': 0})
new_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,1,C_ID_415bb3a509,107,0,1,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,9,19,1,0,0,0,0,0,1,0
1,1,C_ID_415bb3a509,140,0,1,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,9,19,1,0,0,0,0,0,1,0
2,1,C_ID_415bb3a509,330,0,1,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,9,14,1,0,0,0,0,0,1,0
3,1,C_ID_415bb3a509,-1,1,1,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,-1,8,0,0,0,0,0,0,1,0
4,1,C_ID_ef55cf8d4b,-1,1,1,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,-1,29,0,0,0,0,0,0,1,0


In [14]:
merch_new = aggregate_transactions(new_trans, prefix='new_')
del new_trans
gc.collect()
train = pd.merge(train, merch_new, on='card_id',how='left')
test = pd.merge(test, merch_new, on='card_id',how='left')

train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,month,year,elapsed_time,hist_transactions_count,...,new_purchase_amount_min,new_purchase_amount_std,new_installments_sum,new_installments_mean,new_installments_max,new_installments_min,new_installments_std,new_purchase_date_ptp,new_month_lag_min,new_month_lag_max
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,6,2017,245,260,...,-0.724368,0.135812,0.0,0.000000,0.0,0.0,0.000000,4742309.0,1.0,2.0
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,1,2017,396,350,...,-0.739410,0.014326,6.0,1.000000,1.0,1.0,0.000000,4887632.0,1.0,2.0
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,8,2016,549,43,...,-0.700326,NaN,0.0,0.000000,0.0,0.0,NaN,0.0,2.0,2.0
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,9,2017,153,77,...,-0.734135,0.065882,5.0,0.714286,1.0,-1.0,0.755929,3625505.0,1.0,2.0
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,11,2017,92,133,...,-0.739395,0.223821,35.0,0.972222,2.0,-1.0,0.376913,4949682.0,1.0,2.0


In [15]:
del merch_new
gc.collect()

42

In [16]:
cat_cols = ['feature_1', 'feature_2', 'feature_3', 'month', 'year','hist_merchant_id_nunique']

In [17]:
target = train['target']
drops = ['card_id', 'first_active_month', 'target']
num_cols = [col for col in train.columns if col not in cat_cols and col not in drops]
total_cols = [col for col in train.columns]


In [18]:
from sklearn.preprocessing import LabelEncoder
max_values = {}
for col in cat_cols:
    print(col)
    lbl = LabelEncoder()
    lbl.fit(list(train[col].values.astype('str')) + list(test[col].values.astype('str')))
    train[col] = lbl.transform(list(train[col].values.astype('str')))
    test[col] = lbl.transform(list(test[col].values.astype('str')))
    max_values[col] = max(train[col].max(), test[col].max())  + 2
    print(max_values[col])

feature_1
6
feature_2
4
feature_3
3
month
26
year
18
hist_merchant_id_nunique
333


In [19]:
# printing because I'm too lazy to write everything by hand. Open output to see.
for col in cat_cols:
    n = col.replace('.', '_')
    print(f'{n} = Input(shape=[1], name="{col}")')
    print(f'emb_{n} = Embedding({max_values[col]}, {(np.min(max_values[col]+1)//2, 50)})({col})')
    print(',', n)

feature_1 = Input(shape=[1], name="feature_1")
emb_feature_1 = Embedding(6, (3, 50))(feature_1)
, feature_1
feature_2 = Input(shape=[1], name="feature_2")
emb_feature_2 = Embedding(4, (2, 50))(feature_2)
, feature_2
feature_3 = Input(shape=[1], name="feature_3")
emb_feature_3 = Embedding(3, (2, 50))(feature_3)
, feature_3
month = Input(shape=[1], name="month")
emb_month = Embedding(26, (13, 50))(month)
, month
year = Input(shape=[1], name="year")
emb_year = Embedding(18, (9, 50))(year)
, year
hist_merchant_id_nunique = Input(shape=[1], name="hist_merchant_id_nunique")
emb_hist_merchant_id_nunique = Embedding(333, (167, 50))(hist_merchant_id_nunique)
, hist_merchant_id_nunique


In [20]:
#filter num values
target = train['target']
drops = ['card_id', 'first_active_month', 'target']
use_cols_num = [c for c in train.columns if c in num_cols]
features_num = list(train[use_cols_num].columns)
train[features_num].head()

,elapsed_time,hist_transactions_count,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_category_1_mean,hist_category_2_1.0_mean,hist_category_2_2.0_mean,hist_category_2_3.0_mean,hist_category_2_4.0_mean,hist_category_2_5.0_mean,...,new_purchase_amount_min,new_purchase_amount_std,new_installments_sum,new_installments_mean,new_installments_max,new_installments_min,new_installments_std,new_purchase_date_ptp,new_month_lag_min,new_month_lag_max
0,245,260,247,0.950000,0.000000,0.988462,0.0,0.0,0.000000,0.011538,...,-0.724368,0.135812,0.0,0.000000,0.0,0.0,0.000000,4742309.0,1.0,2.0
1,396,350,339,0.968571,0.088571,0.911429,0.0,0.0,0.000000,0.000000,...,-0.739410,0.014326,6.0,1.000000,1.0,1.0,0.000000,4887632.0,1.0,2.0
2,549,43,41,0.953488,0.000000,0.093023,0.0,0.0,0.000000,0.906977,...,-0.700326,NaN,0.0,0.000000,0.0,0.0,NaN,0.0,2.0,2.0
3,153,77,77,1.000000,0.155844,0.155844,0.0,0.0,0.688312,0.000000,...,-0.734135,0.065882,5.0,0.714286,1.0,-1.0,0.755929,3625505.0,1.0,2.0
4,92,133,128,0.962406,0.112782,0.075188,0.0,0.0,0.804511,0.007519,...,-0.739395,0.223821,35.0,0.972222,2.0,-1.0,0.376913,4949682.0,1.0,2.0


In [21]:
#scale the data and impute the null values 
#note: apparently, GPU environment doesn't have an updated version of sklearn,
#so we cannot use sklearn.impute.SimpleImputer. In CPU environement this is possible
from sklearn.preprocessing import StandardScaler, Imputer
sc = StandardScaler()
train[features_num] = train[features_num].fillna(0)
train[features_num] = sc.fit_transform(train[features_num])

In [22]:
#fit test data
test[features_num] = test[features_num].fillna(0)
test[features_num] = sc.fit_transform(test[features_num])

In [23]:
#drop columns
X_train = train.drop([col for col in drops if col in train.columns], axis=1)
X_test = test.drop([col for col in drops if col in test.columns], axis=1)
X_train.shape

(201917, 58)

In [24]:
#renommer certaines colonnes
X_train.rename(columns={'hist_category_2_1.0_mean': 'hist_category_2_1_0_mean',
                        'hist_category_2_2.0_mean': 'hist_category_2_2_0_mean',
                        'hist_category_2_3.0_mean': 'hist_category_2_3_0_mean',
                        'hist_category_2_4.0_mean': 'hist_category_2_4_0_mean',
                        'hist_category_2_5.0_mean': 'hist_category_2_5_0_mean',
                        'new_category_2_1.0_mean': 'new_category_2_1_0_mean',
                        'new_category_2_2.0_mean': 'new_category_2_2_0_mean',
                        'new_category_2_3.0_mean': 'new_category_2_3_0_mean',
                        'new_category_2_4.0_mean': 'new_category_2_4_0_mean',
                        'new_category_2_5.0_mean': 'new_category_2_5_0_mean'}, inplace=True)
total_cols = [col for col in X_train.columns]
total_cols

['feature_1',
 'feature_2',
 'feature_3',
 'month',
 'year',
 'elapsed_time',
 'hist_transactions_count',
 'hist_authorized_flag_sum',
 'hist_authorized_flag_mean',
 'hist_category_1_mean',
 'hist_category_2_1_0_mean',
 'hist_category_2_2_0_mean',
 'hist_category_2_3_0_mean',
 'hist_category_2_4_0_mean',
 'hist_category_2_5_0_mean',
 'hist_category_3_A_mean',
 'hist_category_3_B_mean',
 'hist_category_3_C_mean',
 'hist_merchant_id_nunique',
 'hist_purchase_amount_sum',
 'hist_purchase_amount_mean',
 'hist_purchase_amount_max',
 'hist_purchase_amount_min',
 'hist_purchase_amount_std',
 'hist_installments_sum',
 'hist_installments_mean',
 'hist_installments_max',
 'hist_installments_min',
 'hist_installments_std',
 'hist_purchase_date_ptp',
 'hist_month_lag_min',
 'hist_month_lag_max',
 'new_transactions_count',
 'new_authorized_flag_sum',
 'new_authorized_flag_mean',
 'new_category_1_mean',
 'new_category_2_1_0_mean',
 'new_category_2_2_0_mean',
 'new_category_2_3_0_mean',
 'new_categor

In [25]:
#renommer certaines colonnes
X_test.rename(columns={'hist_category_2_1.0_mean': 'hist_category_2_1_0_mean',
                        'hist_category_2_2.0_mean': 'hist_category_2_2_0_mean',
                        'hist_category_2_3.0_mean': 'hist_category_2_3_0_mean',
                        'hist_category_2_4.0_mean': 'hist_category_2_4_0_mean',
                        'hist_category_2_5.0_mean': 'hist_category_2_5_0_mean',
                        'new_category_2_1.0_mean': 'new_category_2_1_0_mean',
                        'new_category_2_2.0_mean': 'new_category_2_2_0_mean',
                        'new_category_2_3.0_mean': 'new_category_2_3_0_mean',
                        'new_category_2_4.0_mean': 'new_category_2_4_0_mean',
                        'new_category_2_5.0_mean': 'new_category_2_5_0_mean'}, inplace=True)
total_cols = [col for col in X_test.columns]
total_cols

['feature_1',
 'feature_2',
 'feature_3',
 'month',
 'year',
 'elapsed_time',
 'hist_transactions_count',
 'hist_authorized_flag_sum',
 'hist_authorized_flag_mean',
 'hist_category_1_mean',
 'hist_category_2_1_0_mean',
 'hist_category_2_2_0_mean',
 'hist_category_2_3_0_mean',
 'hist_category_2_4_0_mean',
 'hist_category_2_5_0_mean',
 'hist_category_3_A_mean',
 'hist_category_3_B_mean',
 'hist_category_3_C_mean',
 'hist_merchant_id_nunique',
 'hist_purchase_amount_sum',
 'hist_purchase_amount_mean',
 'hist_purchase_amount_max',
 'hist_purchase_amount_min',
 'hist_purchase_amount_std',
 'hist_installments_sum',
 'hist_installments_mean',
 'hist_installments_max',
 'hist_installments_min',
 'hist_installments_std',
 'hist_purchase_date_ptp',
 'hist_month_lag_min',
 'hist_month_lag_max',
 'new_transactions_count',
 'new_authorized_flag_sum',
 'new_authorized_flag_mean',
 'new_category_1_mean',
 'new_category_2_1_0_mean',
 'new_category_2_2_0_mean',
 'new_category_2_3_0_mean',
 'new_categor

In [26]:
num_cols = [col for col in X_train.columns if col not in cat_cols and col not in drops]
num_cols

['elapsed_time',
 'hist_transactions_count',
 'hist_authorized_flag_sum',
 'hist_authorized_flag_mean',
 'hist_category_1_mean',
 'hist_category_2_1_0_mean',
 'hist_category_2_2_0_mean',
 'hist_category_2_3_0_mean',
 'hist_category_2_4_0_mean',
 'hist_category_2_5_0_mean',
 'hist_category_3_A_mean',
 'hist_category_3_B_mean',
 'hist_category_3_C_mean',
 'hist_purchase_amount_sum',
 'hist_purchase_amount_mean',
 'hist_purchase_amount_max',
 'hist_purchase_amount_min',
 'hist_purchase_amount_std',
 'hist_installments_sum',
 'hist_installments_mean',
 'hist_installments_max',
 'hist_installments_min',
 'hist_installments_std',
 'hist_purchase_date_ptp',
 'hist_month_lag_min',
 'hist_month_lag_max',
 'new_transactions_count',
 'new_authorized_flag_sum',
 'new_authorized_flag_mean',
 'new_category_1_mean',
 'new_category_2_1_0_mean',
 'new_category_2_2_0_mean',
 'new_category_2_3_0_mean',
 'new_category_2_4_0_mean',
 'new_category_2_5_0_mean',
 'new_category_3_A_mean',
 'new_category_3_B_me

In [37]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Dense, BatchNormalization, Dropout, Input
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from sklearn.model_selection import KFold, StratifiedKFold
train_y = target.values


In [28]:
from sklearn.model_selection import TimeSeriesSplit, KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from keras.models import Model, load_model
from keras.layers import Input, Dropout, Dense, Embedding, SpatialDropout1D, concatenate, BatchNormalization, Flatten

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.models import Model
from keras.losses import mean_squared_error as mse_loss

from keras import optimizers
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#building the network

from keras.initializers import he_normal, he_uniform,  glorot_normal,  glorot_uniform
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D

from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D


In [29]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=0))

In [30]:
def model(dense_dim_1=128, dense_dim_2=64, dense_dim_3=256, dense_dim_4=512,  dense_dim_5=512, dense_dim_6=256,
dropout1=0.1, dropout2=0.1, dropout3=0.2, dropout4=0.2, dropout5=0.2, dropout6=0.2, lr=0.0001):

    #Inputs cat
    feature_1 = Input(shape=[1], name="feature_1")
    feature_2 = Input(shape=[1], name="feature_2")
    feature_3 = Input(shape=[1], name="feature_3")
    month = Input(shape=[1], name="month")
    year = Input(shape=[1], name="year")
    hist_merchant_id_nunique = Input(shape=[1], name="hist_merchant_id_nunique")
    
    #Input num    
    elapsed_time = Input(shape=[1], name="elapsed_time")
    hist_transactions_count = Input(shape=[1], name="hist_transactions_count")
    hist_authorized_flag_sum = Input(shape=[1], name="hist_authorized_flag_sum")
    hist_authorized_flag_mean = Input(shape=[1], name="hist_authorized_flag_mean")
    hist_category_1_mean = Input(shape=[1], name="hist_category_1_mean")
    hist_category_2_1_0_mean = Input(shape=[1], name="hist_category_2_1_0_mean")
    hist_category_2_2_0_mean = Input(shape=[1], name="hist_category_2_2_0_mean")
    hist_category_2_3_0_mean = Input(shape=[1], name="hist_category_2_3_0_mean")
    hist_category_2_4_0_mean = Input(shape=[1], name="hist_category_2_4_0_mean")
    hist_category_2_5_0_mean = Input(shape=[1], name="hist_category_2_5_0_mean")
    hist_category_3_A_mean = Input(shape=[1], name="hist_category_3_A_mean")
    hist_category_3_B_mean = Input(shape=[1], name="hist_category_3_B_mean")
    hist_category_3_C_mean = Input(shape=[1], name="hist_category_3_C_mean")
    hist_purchase_amount_sum = Input(shape=[1], name="hist_purchase_amount_sum")
    hist_purchase_amount_mean = Input(shape=[1], name="hist_purchase_amount_mean")
    hist_purchase_amount_max = Input(shape=[1], name="hist_purchase_amount_max")
    hist_purchase_amount_min = Input(shape=[1], name="hist_purchase_amount_min")
    hist_purchase_amount_std = Input(shape=[1], name="hist_purchase_amount_std")
    hist_installments_sum = Input(shape=[1], name="hist_installments_sum")
    hist_installments_mean = Input(shape=[1], name="hist_installments_mean")
    hist_installments_max = Input(shape=[1], name="hist_installments_max")
    hist_installments_min = Input(shape=[1], name="hist_installments_min")
    hist_installments_std = Input(shape=[1], name="hist_installments_std")
    hist_purchase_date_ptp = Input(shape=[1], name="hist_purchase_date_ptp")
    hist_month_lag_min = Input(shape=[1], name="hist_month_lag_min")
    hist_month_lag_max = Input(shape=[1], name="hist_month_lag_max")
    
    new_transactions_count = Input(shape=[1], name="new_transactions_count")
    new_authorized_flag_sum = Input(shape=[1], name="new_authorized_flag_sum")
    new_authorized_flag_mean = Input(shape=[1], name="new_authorized_flag_mean")
    new_category_1_mean = Input(shape=[1], name="new_category_1_mean")
    new_category_2_1_0_mean = Input(shape=[1], name="new_category_2_1_0_mean")
    new_category_2_2_0_mean = Input(shape=[1], name="new_category_2_2_0_mean")
    new_category_2_3_0_mean = Input(shape=[1], name="new_category_2_3_0_mean")
    new_category_2_4_0_mean = Input(shape=[1], name="new_category_2_4_0_mean")
    new_category_2_5_0_mean = Input(shape=[1], name="new_category_2_5_0_mean")
    new_category_3_A_mean = Input(shape=[1], name="new_category_3_A_mean")
    new_category_3_B_mean = Input(shape=[1], name="new_category_3_B_mean")
    new_category_3_C_mean = Input(shape=[1], name="new_category_3_C_mean")
    new_purchase_amount_sum = Input(shape=[1], name="new_purchase_amount_sum")
    new_purchase_amount_mean = Input(shape=[1], name="new_purchase_amount_mean")
    new_purchase_amount_max = Input(shape=[1], name="new_purchase_amount_max")
    new_purchase_amount_min = Input(shape=[1], name="new_purchase_amount_min")
    new_purchase_amount_std = Input(shape=[1], name="new_purchase_amount_std")
    new_installments_sum = Input(shape=[1], name="new_installments_sum")
    new_installments_mean = Input(shape=[1], name="new_installments_mean")
    new_installments_max = Input(shape=[1], name="new_installments_max")
    new_installments_min = Input(shape=[1], name="new_installments_min")
    new_installments_std = Input(shape=[1], name="new_installments_std")
    new_purchase_date_ptp = Input(shape=[1], name="new_purchase_date_ptp")
    new_month_lag_min = Input(shape=[1], name="new_month_lag_min")
    new_month_lag_max = Input(shape=[1], name="new_month_lag_max")    
    
    #Embeddings layers

    emb_feature_1 = Embedding(6, 3)(feature_1)
    emb_feature_2 = Embedding(4, 3)(feature_2)
    emb_feature_3 = Embedding(4, 3)(feature_3)
    emb_month = Embedding(26, 13)(month)
    emb_year = Embedding(18, 9)(year)
    emb_hist_merchant_id_nunique = Embedding(333, 50)(hist_merchant_id_nunique)


    concat_emb1 = concatenate([
           Flatten() (emb_feature_1),
            Flatten() (emb_feature_2),
            Flatten() (emb_feature_3),
            Flatten() (emb_month),
            Flatten() (emb_year),
            Flatten() (emb_hist_merchant_id_nunique)
    ])
    
    categ = Dropout(dropout1)(Dense(dense_dim_1,kernel_initializer=he_uniform(seed=SEED),activation='relu') (concat_emb1))
    categ = BatchNormalization()(categ)
    categ = Dropout(dropout2)(Dense(dense_dim_2,kernel_initializer=he_uniform(seed=SEED),activation='relu') (categ))
        
    #main layer
    main_l = concatenate([
          categ
        , elapsed_time
        , hist_transactions_count
        , hist_authorized_flag_sum
        , hist_authorized_flag_mean
        , hist_category_1_mean
        , hist_category_2_1_0_mean
        , hist_category_2_2_0_mean
        , hist_category_2_3_0_mean
        , hist_category_2_4_0_mean 
        , hist_category_2_5_0_mean
        , hist_category_3_A_mean
        , hist_category_3_B_mean
        , hist_category_3_C_mean
        , hist_purchase_amount_sum
        , hist_purchase_amount_mean
        , hist_purchase_amount_max
        , hist_purchase_amount_min
        , hist_purchase_amount_std
        , hist_installments_sum
        , hist_installments_mean
        , hist_installments_max
        , hist_installments_min
        , hist_installments_std 
        , hist_purchase_date_ptp
        , hist_month_lag_min
        , hist_month_lag_max
        , new_transactions_count
        , new_authorized_flag_sum
        , new_authorized_flag_mean
        , new_category_1_mean
        , new_category_2_1_0_mean
        , new_category_2_2_0_mean
        , new_category_2_3_0_mean
        , new_category_2_4_0_mean 
        , new_category_2_5_0_mean
        , new_category_3_A_mean
        , new_category_3_B_mean
        , new_category_3_C_mean
        , new_purchase_amount_sum
        , new_purchase_amount_mean
        , new_purchase_amount_max
        , new_purchase_amount_min
        , new_purchase_amount_std
        , new_installments_sum
        , new_installments_mean
        , new_installments_max
        , new_installments_min
        , new_installments_std 
        , new_purchase_date_ptp
        , new_month_lag_min
        , new_month_lag_max
    ])
    
    main_l = Dropout(dropout3)(Dense(dense_dim_3,kernel_initializer=he_uniform(seed=SEED),activation='relu') (main_l))
    main_l = BatchNormalization()(main_l)
    main_l = Dropout(dropout4)(Dense(dense_dim_4,kernel_initializer=he_uniform(seed=SEED),activation='relu') (main_l))
    main_l = BatchNormalization()(main_l)
    main_l = Dropout(dropout5)(Dense(dense_dim_5,kernel_initializer=he_uniform(seed=SEED),activation='relu') (main_l))
    main_l = BatchNormalization()(main_l) 
    main_l = Dropout(dropout6)(Dense(dense_dim_6,kernel_initializer=he_uniform(seed=SEED),activation='relu') (main_l))

    
    #output
    output = Dense(1) (main_l)

    model = Model([feature_1,
                    feature_2,
                    feature_3,
                    month,
                    year,
                    hist_merchant_id_nunique,
                    elapsed_time,
                    hist_transactions_count,
                    hist_authorized_flag_sum,
                    hist_authorized_flag_mean,
                    hist_category_1_mean,
                    hist_category_2_1_0_mean,
                    hist_category_2_2_0_mean,
                    hist_category_2_3_0_mean,
                    hist_category_2_4_0_mean, 
                    hist_category_2_5_0_mean,
                    hist_category_3_A_mean,
                    hist_category_3_B_mean,
                    hist_category_3_C_mean,
                    hist_purchase_amount_sum,
                    hist_purchase_amount_mean,
                    hist_purchase_amount_max,
                    hist_purchase_amount_min,
                    hist_purchase_amount_std,
                    hist_installments_sum,
                    hist_installments_mean,
                    hist_installments_max,
                    hist_installments_min,
                    hist_installments_std, 
                    hist_purchase_date_ptp,
                    hist_month_lag_min,
                    hist_month_lag_max,
                    new_transactions_count,
                    new_authorized_flag_sum,
                    new_authorized_flag_mean,
                    new_category_1_mean,
                    new_category_2_1_0_mean,
                    new_category_2_2_0_mean,
                    new_category_2_3_0_mean,
                    new_category_2_4_0_mean, 
                    new_category_2_5_0_mean,
                    new_category_3_A_mean,
                    new_category_3_B_mean,
                    new_category_3_C_mean,
                    new_purchase_amount_sum,
                    new_purchase_amount_mean,
                    new_purchase_amount_max,
                    new_purchase_amount_min,
                    new_purchase_amount_std,
                    new_installments_sum,
                    new_installments_mean,
                    new_installments_max,
                    new_installments_min,
                    new_installments_std, 
                    new_purchase_date_ptp,
                    new_month_lag_min,
                    new_month_lag_max], output)

    #model = Model([**params], output)
    model.compile(optimizer = Adam(lr=lr),
                  loss= rmse,
                  metrics=[rmse])
    return model


#model=model()

In [31]:
# converting data to format which can be used by Keras
def get_keras_data(df, num_cols, cat_cols):
    cols = num_cols + cat_cols

    X = {col: np.array(df[col]) for col in cols}
    # print("Data ready for Vectorization")
    
    return X

In [32]:
X_test_keras = get_keras_data(X_test, num_cols, cat_cols)


In [33]:
def train_model(keras_model, X_t, y_train, batch_size, epochs, X_v, y_valid, reduce_lr=True, patience=7):
    """
    Helper function to train model. Also I noticed that ReduceLROnPlateau is rarely
    useful, so added an option to turn it off.
    """
    
    early_stopping = EarlyStopping(patience=patience, verbose=1)
    model_checkpoint = ModelCheckpoint("model.hdf5",
                                       save_best_only=True, verbose=1, monitor='val_loss', mode='min')
    if reduce_lr:
        reduce_lr = ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.000005, verbose=1)
        hist = keras_model.fit(X_t, y_train, batch_size=batch_size, epochs=epochs,
                            validation_data=(X_v, y_valid), verbose=True,
                            callbacks=[early_stopping, model_checkpoint, reduce_lr])
    
    else:
        hist = keras_model.fit(X_t, y_train, batch_size=batch_size, epochs=epochs,
                            validation_data=(X_v, y_valid), verbose=True,
                            callbacks=[early_stopping, model_checkpoint])

    
    return hist

In [34]:
X_train['outliers'] = 0
X_train.loc[target < -30, 'outliers'] = 1
X_train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [42]:
batch_size = 256
epochs = 100
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)
oof = np.zeros(len(train))
predictions = np.zeros(len(test))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train.values, X_train["outliers"].values)):
    keras_model = model()
    X_t = get_keras_data(X_train.iloc[trn_idx], num_cols, cat_cols)
    X_v = get_keras_data(X_train.iloc[val_idx], num_cols, cat_cols)
    y_train = train_y[trn_idx]
    y_val = train_y[val_idx]
    hist = train_model(keras_model, X_t, y_train, batch_size, epochs, X_v, y_val, reduce_lr=True, patience=7)
    keras_modelmax = load_model("model.hdf5", custom_objects={'rmse': rmse})
    oof[val_idx] = keras_modelmax.predict(X_v)
    predictions += keras_modelmax.predict(X_test_keras) / folds.n_splits

Train on 161533 samples, validate on 40384 samples
Epoch 1/100
161533/161533 [==============================] - 42s 258us/step - loss: 3.8533 - rmse: 3.8533 - val_loss: 3.7319 - val_rmse: 3.7319

Epoch 00001: val_loss improved from inf to 3.73194, saving model to model.hdf5
Epoch 2/100
161533/161533 [==============================] - 14s 89us/step - loss: 3.7706 - rmse: 3.7706 - val_loss: 3.7130 - val_rmse: 3.7130

Epoch 00002: val_loss improved from 3.73194 to 3.71296, saving model to model.hdf5
Epoch 3/100
161533/161533 [==============================] - 14s 90us/step - loss: 3.7489 - rmse: 3.7489 - val_loss: 3.7056 - val_rmse: 3.7056

Epoch 00003: val_loss improved from 3.71296 to 3.70564, saving model to model.hdf5
Epoch 4/100
161533/161533 [==============================] - 15s 92us/step - loss: 3.7252 - rmse: 3.7252 - val_loss: 3.7067 - val_rmse: 3.7067

Epoch 00004: val_loss did not improve from 3.70564
Epoch 5/100
161533/161533 [==============================] - 15s 91us/step -

161533/161533 [==============================] - 15s 91us/step - loss: 3.6617 - rmse: 3.6617 - val_loss: 3.6736 - val_rmse: 3.6736

Epoch 00011: val_loss improved from 3.67605 to 3.67360, saving model to model.hdf5
Epoch 12/100
161533/161533 [==============================] - 15s 91us/step - loss: 3.6540 - rmse: 3.6540 - val_loss: 3.6717 - val_rmse: 3.6717

Epoch 00012: val_loss improved from 3.67360 to 3.67167, saving model to model.hdf5
Epoch 13/100
161533/161533 [==============================] - 15s 92us/step - loss: 3.6411 - rmse: 3.6411 - val_loss: 3.6729 - val_rmse: 3.6729

Epoch 00013: val_loss did not improve from 3.67167
Epoch 14/100
161533/161533 [==============================] - 15s 91us/step - loss: 3.6224 - rmse: 3.6224 - val_loss: 3.6684 - val_rmse: 3.6684

Epoch 00014: val_loss improved from 3.67167 to 3.66839, saving model to model.hdf5
Epoch 15/100
161533/161533 [==============================] - 15s 90us/step - loss: 3.6218 - rmse: 3.6218 - val_loss: 3.6649 - val_rm

161534/161534 [==============================] - 15s 93us/step - loss: 3.7472 - rmse: 3.7472 - val_loss: 3.6845 - val_rmse: 3.6845

Epoch 00003: val_loss improved from 3.69680 to 3.68454, saving model to model.hdf5
Epoch 4/100
161534/161534 [==============================] - 15s 92us/step - loss: 3.7261 - rmse: 3.7261 - val_loss: 3.6807 - val_rmse: 3.6807

Epoch 00004: val_loss improved from 3.68454 to 3.68069, saving model to model.hdf5
Epoch 5/100
161534/161534 [==============================] - 15s 92us/step - loss: 3.7137 - rmse: 3.7137 - val_loss: 3.6723 - val_rmse: 3.6723

Epoch 00005: val_loss improved from 3.68069 to 3.67234, saving model to model.hdf5
Epoch 6/100
161534/161534 [==============================] - 15s 92us/step - loss: 3.6989 - rmse: 3.6989 - val_loss: 3.6693 - val_rmse: 3.6693

Epoch 00006: val_loss improved from 3.67234 to 3.66933, saving model to model.hdf5
Epoch 7/100
161534/161534 [==============================] - 15s 93us/step - loss: 3.6883 - rmse: 3.6883

161534/161534 [==============================] - 15s 91us/step - loss: 3.6172 - rmse: 3.6172 - val_loss: 3.6394 - val_rmse: 3.6394

Epoch 00015: val_loss improved from 3.64029 to 3.63944, saving model to model.hdf5
Epoch 16/100
161534/161534 [==============================] - 15s 90us/step - loss: 3.6227 - rmse: 3.6227 - val_loss: 3.6409 - val_rmse: 3.6409

Epoch 00016: val_loss did not improve from 3.63944
Epoch 17/100
161534/161534 [==============================] - 15s 90us/step - loss: 3.6281 - rmse: 3.6281 - val_loss: 3.6398 - val_rmse: 3.6398

Epoch 00017: val_loss did not improve from 3.63944

Epoch 00017: ReduceLROnPlateau reducing learning rate to 5e-06.
Epoch 18/100
161534/161534 [==============================] - 15s 90us/step - loss: 3.6310 - rmse: 3.6310 - val_loss: 3.6413 - val_rmse: 3.6413

Epoch 00018: val_loss did not improve from 3.63944
Epoch 19/100
161534/161534 [==============================] - 15s 90us/step - loss: 3.6214 - rmse: 3.6214 - val_loss: 3.6398 - val_r

In [43]:
valid_score = mean_squared_error(oof,train_y)** 0.5
print("Validation score: ", valid_score)

Validation score:  3.8705589161316296


In [73]:
#saving the card_ids
ids = test['card_id'].values
submission = pd.DataFrame(ids, columns=['card_id'])

In [45]:
submission['target'] = predictions
submission.head(10)

,card_id,target
0,C_ID_0ab67a22ab,0.0
1,C_ID_130fd0cbdd,0.0
2,C_ID_b709037bc5,0.0
3,C_ID_d27d835a9f,0.0
4,C_ID_2b5e3df5c2,0.0
5,C_ID_5814b4f13c,0.0
6,C_ID_a1b3c75277,0.0
7,C_ID_f7cada36d3,0.0
8,C_ID_9d2bc8dfc4,0.0
9,C_ID_6d8dba8475,0.0


In [46]:
submission.to_csv("submission_neuralnet.csv", index = False, header = True)

In [47]:
print("done")

done


In [64]:
inter_model = Model(input = keras_model.input, output = keras_model.get_layer("concatenate_9").output)

In [66]:
X_t = get_keras_data(X_train, num_cols, cat_cols)
embed_pred_test = inter_model.predict(X_test_keras)
embed_pred_train = inter_model.predict(X_t)

In [68]:
np.save("embed_pred_test", embed_pred_test)
np.save("embed_pred_train", embed_pred_train)

In [75]:
submission['embed'] = embed_pred_test.tolist()

In [76]:
submission.to_csv("test_embed.csv", index = False, header = True)

In [72]:
ids = train['card_id'].values
train_embed = pd.DataFrame(ids, columns=['card_id'])
train_embed['embed'] = embed_pred_train.tolist()
train_embed.to_csv("train_embed.csv", index = False, header = True)